<h3>Question 1: Toronto Neighborhood Data</h3>

In [1]:
#load up pandas for the table scrape
import pandas as pd

<b>Section 1: Collect the data</b>

In [2]:
#scrape the table from the wiki into a dataframe, headers in first row

table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)

df = table[0]

In [3]:
#peek at our df to make sure it worked
print(df.head())
print(df.shape)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
(287, 3)


<b> Section 2: Remove 'Not Assigned' Boroughs and rename 'Not Assigned' Neighbourhood(s)</b>

In [4]:
#drop the Borough rows that are "Not assigned" based on their row index
drop_indices = df[df['Borough'] == 'Not assigned'].index

df.drop(drop_indices, inplace=True)

#reset the index to have a meaningful sequence instead of gaps
df.reset_index(drop=True, inplace=True)

#see if it worked by checking the head/shape again
print(df.head())
print(df.shape)

  Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M6A        North York  Lawrence Heights
4      M6A        North York    Lawrence Manor
(210, 3)


In [5]:
#find the leftover 'Not assigned' Neighbourhood(s)
df.loc[df['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
6,M9A,Queen's Park,Not assigned


In [6]:
#using the index above, rename the 'Not assigned' to the Borough name
df['Neighbourhood'][6] = "Queen's Park"

#make sure it worked
df.loc[df['Borough'] == "Queen's Park"]

,Postcode,Borough,Neighbourhood
6,M9A,Queen's Park,Queen's Park


<b>Section 3: Merge the Postcode multiples</b>

In [7]:
#group the rows by 'Postcode,' then by 'Borough.'
#combine the 'Neighbourhood' multiples, comma-separated, reassigned to df.
#reset the index to start from 0.

df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

In [8]:
print(df.head())
print("Final shape of dataframe: ", df.shape)

  Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae
Final shape of dataframe:  (103, 3)


<h3>Question 2: Latitude & Longitude</h3>

<b>Method 1: Geocoder</b>

In [ ]:
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

<b>Method 2: Import data from CSV provided</b>

In [10]:
df_coord = pd.read_csv("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")

In [11]:
df_coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
#join the original df to the Longitude/Latitude df
df_toronto = pd.concat([df,df_coord], axis=1)

In [13]:
#check for line-up on Postcode/Postal Code
display(df_toronto)

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",M9R,43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",M9V,43.739416,-79.588437


In [14]:
#drop the redundant Postal Code column
df_toronto.drop(['Postal Code'], axis=1)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


<h3>Question 3: Explore the Data</h3>

<b>Visualization</b>

In [ ]:
#libraries referenced in New York Lab
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
!conda install -c anaconda scikit-learn
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='cadetblue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork